# Entraîner un RL à équilibrer un Cartpole

Ce notebook fait partie du [programme AI for Beginners](http://aka.ms/ai-beginners). Il s'inspire du [tutoriel officiel de PyTorch](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) et de [cette implémentation Cartpole avec PyTorch](https://github.com/yc930401/Actor-Critic-pytorch).

Dans cet exemple, nous utiliserons le RL pour entraîner un modèle à équilibrer une barre sur un chariot qui peut se déplacer à gauche et à droite sur une échelle horizontale. Nous utiliserons l'environnement [OpenAI Gym](https://www.gymlibrary.ml/) pour simuler la barre.

> **Note** : Vous pouvez exécuter le code de cette leçon localement (par exemple, depuis Visual Studio Code), auquel cas la simulation s'ouvrira dans une nouvelle fenêtre. Lorsque vous exécutez le code en ligne, il peut être nécessaire d'apporter quelques ajustements au code, comme décrit [ici](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

Nous commencerons par nous assurer que Gym est installé :


In [ ]:
import sys
!{sys.executable} -m pip install gym

Créons maintenant l'environnement CartPole et voyons comment l'utiliser. Un environnement possède les propriétés suivantes :

* **Action space** est l'ensemble des actions possibles que nous pouvons effectuer à chaque étape de la simulation  
* **Observation space** est l'ensemble des observations que nous pouvons réaliser  


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Voyons comment fonctionne la simulation. La boucle suivante exécute la simulation jusqu'à ce que `env.step` ne renvoie plus le drapeau de terminaison `done`. Nous choisirons des actions de manière aléatoire en utilisant `env.action_space.sample()`, ce qui signifie que l'expérience échouera probablement très rapidement (l'environnement CartPole se termine lorsque la vitesse du CartPole, sa position ou son angle dépassent certaines limites).

> La simulation s'ouvrira dans une nouvelle fenêtre. Vous pouvez exécuter le code plusieurs fois et observer son comportement.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Vous pouvez remarquer que les observations contiennent 4 nombres. Ce sont :
- Position du chariot
- Vitesse du chariot
- Angle de la tige
- Taux de rotation de la tige

`rew` est la récompense que nous recevons à chaque étape. Vous pouvez constater que dans l'environnement CartPole, vous recevez 1 point de récompense pour chaque étape de simulation, et l'objectif est de maximiser la récompense totale, c'est-à-dire le temps pendant lequel le CartPole peut rester en équilibre sans tomber.

Pendant l'apprentissage par renforcement, notre objectif est d'entraîner une **politique** $\pi$, qui pour chaque état $s$ nous indiquera quelle action $a$ entreprendre, donc essentiellement $a = \pi(s)$.

Si vous souhaitez une solution probabiliste, vous pouvez considérer la politique comme renvoyant un ensemble de probabilités pour chaque action, c'est-à-dire que $\pi(a|s)$ représenterait la probabilité que nous devrions entreprendre l'action $a$ dans l'état $s$.

## Méthode du Gradient de Politique

Dans l'algorithme d'apprentissage par renforcement le plus simple, appelé **Gradient de Politique**, nous allons entraîner un réseau de neurones à prédire la prochaine action.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Nous allons entraîner le réseau en réalisant de nombreuses expériences et en mettant à jour notre réseau après chaque exécution. Définissons une fonction qui exécutera l'expérience et renverra les résultats (le **trace** ainsi nommé) - tous les états, actions (et leurs probabilités recommandées), et récompenses :


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Vous pouvez exécuter un épisode avec un réseau non entraîné et observer que la récompense totale (AKA durée de l'épisode) est très faible :


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

L'un des aspects délicats de l'algorithme de gradient de politique est d'utiliser **les récompenses actualisées**. L'idée est que nous calculons le vecteur des récompenses totales à chaque étape du jeu, et pendant ce processus, nous actualisons les premières récompenses en utilisant un coefficient $gamma$. Nous normalisons également le vecteur résultant, car nous l'utiliserons comme poids pour influencer notre entraînement :


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Passons maintenant à l'entraînement proprement dit ! Nous allons exécuter 300 épisodes, et à chaque épisode, nous effectuerons les étapes suivantes :

1. Exécuter l'expérience et collecter la trace.
2. Calculer la différence (`gradients`) entre les actions effectuées et les probabilités prédites. Plus cette différence est faible, plus nous sommes certains d'avoir pris la bonne décision.
3. Calculer les récompenses actualisées et multiplier les gradients par ces récompenses actualisées - cela garantit que les étapes avec des récompenses plus élevées auront un impact plus important sur le résultat final que celles avec des récompenses plus faibles.
4. Les actions cibles attendues pour notre réseau neuronal seront en partie issues des probabilités prédites pendant l'exécution, et en partie des gradients calculés. Nous utiliserons le paramètre `alpha` pour déterminer dans quelle mesure les gradients et les récompenses sont pris en compte - c'est ce qu'on appelle le *taux d'apprentissage* de l'algorithme de renforcement.
5. Enfin, nous entraînons notre réseau sur les états et les actions attendues, puis nous répétons le processus.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Maintenant, lançons l'épisode avec le rendu pour voir le résultat :


In [ ]:
_ = run_episode(render=True)

Espérons que vous pouvez constater que la tige peut maintenant s'équilibrer assez bien !

## Modèle Acteur-Critique

Le modèle Acteur-Critique est une évolution des gradients de politique, dans lequel nous construisons un réseau neuronal pour apprendre à la fois la politique et les récompenses estimées. Le réseau aura deux sorties (ou vous pouvez le voir comme deux réseaux distincts) :
* **Acteur** recommandera l'action à entreprendre en nous donnant la distribution de probabilité des états, comme dans le modèle de gradient de politique.
* **Critique** estimera quelle serait la récompense issue de ces actions. Il renvoie les récompenses totales estimées dans le futur pour l'état donné.

Définissons un tel modèle :


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Nous devrions légèrement modifier nos fonctions `discounted_rewards` et `run_episode` :


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Maintenant, nous allons exécuter la boucle principale d'entraînement. Nous utiliserons un processus d'entraînement manuel du réseau en calculant les fonctions de perte appropriées et en mettant à jour les paramètres du réseau :


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Points clés

Nous avons vu deux algorithmes de RL dans cette démonstration : le gradient de politique simple et l'acteur-critique plus sophistiqué. Vous pouvez constater que ces algorithmes fonctionnent avec des notions abstraites d'état, d'action et de récompense - ce qui leur permet d'être appliqués à des environnements très différents.

L'apprentissage par renforcement nous permet d'apprendre la meilleure stratégie pour résoudre un problème simplement en observant la récompense finale. Le fait de ne pas avoir besoin de jeux de données étiquetés nous permet de répéter les simulations plusieurs fois afin d'optimiser nos modèles. Cependant, il reste encore de nombreux défis dans le domaine du RL, que vous pourrez découvrir si vous décidez de vous concentrer davantage sur cette branche fascinante de l'IA.



---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction humaine professionnelle. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
